In [2]:
from tensorflow.keras.applications import ResNet50  # Replace with your pre-trained model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense


2024-05-23 11:01:42.460005: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-23 11:01:42.468464: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-23 11:01:42.560702: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-23 11:01:44.894005: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
# Define training and validation data directories
train_data_dir = '/content/gdrive/MyDrive/Snow_Covered_Solar_Panel_Detect/Images/train'
val_data_dir = '/content/gdrive/MyDrive/Snow_Covered_Solar_Panel_Detect/Images/validate'


In [ ]:
FAST_RUN = False
img_width=256
img_height=256
IMAGE_SIZE=(img_width, img_height)
IMAGE_CHANNELS=3
BATCH_SIZE = 32

### ImageDataGenerator and Reference Learning

**ImageDataGenerator** is a powerful tool in Keras for image augmentation, a technique that artificially expands your dataset by creating variations of existing images. This is particularly valuable in reference learning (image classification), where a model learns to distinguish between different categories of objects in images. Here's how ImageDataGenerator aids reference learning:

Reduces Overfitting: Limited training data can lead to overfitting, where a model performs well on the training set but poorly on unseen data. Image augmentation helps by generating diverse image variations, forcing the model to learn more generalizable features that are not specific to the original training images.
Improves Model Robustness: Real-world images encounter variations in lighting, rotation, scaling, and other factors. Image augmentation helps the model learn to be robust to these variations, making it perform better on unseen images that might have different characteristics than those in the training set.
Key ImageDataGenerator Parameters

Here are some essential parameters of ImageDataGenerator to consider for image augmentation:

rescale: Normalizes pixel values between 0 and 1 (commonly used for image classification models).

shear_range: Randomly applies shearing transformation for rotation-like effects.

zoom_range: Randomly zooms in or out of the image to simulate different viewing distances.

horizontal_flip: Randomly flips images horizontally for data augmentation.

rotation_range: Rotates images randomly within a specified degree range.

width_shift_range, height_shift_range: Randomly shifts images horizontally or vertically within a specified range of pixel values.

fill_mode: Defines how to handle pixels beyond the original image boundaries during transformations (e.g., 'nearest', 'constant').

In [ ]:
# Create ImageDataGenerator objects for training and validation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
# Load images using flow_from_directory with data augmentation
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),  # Replace with your image dimensions
    batch_size=32,
    class_mode='categorical'  # Adjust for binary or multi-class classification
)


In [ ]:
# validation data generator
validation_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical'
)


In [ ]:
# Load the pre-trained model (e.g., ResNet50)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))


In [ ]:
# Freeze pre-trained layers (adjust number of layers to freeze as needed)
for layer in base_model.layers[:-2]:
    layer.trainable = False


In [ ]:
# Add custom layers for your specific classification task
x = base_model.output
x = Flatten()(x)
x = Dense(units=1024, activation='relu')(x)  # Adjust number of units for your classes
predictions = Dense(num_classes, activation='softmax')(x)


In [ ]:
# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
# Train the model with fit_generator for data augmentation
history = model.fit(train_generator,
                    epochs=10,  # Adjust number of epochs as needed
                    validation_data=validation_generator)


In [ ]:
# Evaluate the model on unseen data
test_loss, test_acc = model.evaluate(validation_generator)
print('Test accuracy:', test_acc)
